In [1]:
# based on https://flyyufelix.github.io/2016/10/08/fine-tuning-in-keras-part2.html

from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.datasets import cifar10
from keras import backend as K
from keras.utils import np_utils

from sklearn.metrics import log_loss


Using TensorFlow backend.


In [2]:
# load cifar10 data
nb_train_samples = 3000 # 3000 training samples
nb_valid_samples = 100 # 100 validation samples
num_classes = 10

img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 10

def load_cifar10_data(img_rows, img_cols):
    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

    # Resize trainging images
    if K.image_dim_ordering() == 'th':
        X_train = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_valid[:nb_valid_samples,:,:,:]])
    else:
        X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:nb_valid_samples,:,:,:]])

    # Transform targets to keras compatible format
    Y_train = np_utils.to_categorical(Y_train[:nb_train_samples], num_classes)
    Y_valid = np_utils.to_categorical(Y_valid[:nb_valid_samples], num_classes)

    return X_train, Y_train, X_valid, Y_valid


In [17]:
from keras.models import model_from_json, load_model
# temporary value to hold while we debug
i = 0

# 1) decodes the algorithm solution to get hyperparameters
# 2) trains the individual model
# 3) calculates fitness function on validation set and returns fitness score
def train_evaluate(ga_individual_solution):   
    # Load individual solution from json guess format
    print(type(ga_individual_solution))
    pprint.pprint(ga_individual_solution)
    
    # Segment the train_data based on new window_size; split into train and validation (80/20)
    #X,Y = load_cifar10_data(train_data,window_size)
    #X_train, X_val, y_train, y_val = split(X, Y, test_size = 0.20, random_state = 1120)
    
    # Train LSTM model and predict on validation set
    #inputs = Input(shape=(window_size,1))
    #x = LSTM(num_units, input_shape=(window_size,1))(inputs)
    #predictions = Dense(1, activation='linear')(x)
    #model = Model(inputs=inputs, outputs=predictions)
    #model.compile(optimizer='adam',loss='mean_squared_error')
    #model.fit(X_train, y_train, epochs=5, batch_size=10,shuffle=True)
    #y_pred = model.predict(X_val)
    
    # Calculate the RMSE score as fitness score for GA
    #rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    
    global i
    i = i+1
    print('Validation Fitness Function: ', i,'\n')
    
    return i,

In [18]:
import random
import json
import array

from deap import base
from deap import creator
from deap import tools
from deap import algorithms


import pprint

JSON_PATH = "/home/collin/Desktop/EA/evnet/model-json/"

with open(JSON_PATH+"vgg16.json", "r") as network_data:
    network = json.load(network_data)

layer_data = network["layers"]

# We want to maximize accuracy
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, typecode='i', fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Initialize with json file
def initPopulation(pcls, ind_init, filename):
    with open(filename, "r") as pop_file:
        contents = json.load(pop_file)
    return pcls(ind_init(c) for c in contents)

def initIndividual(icls, content):
    return icls(content)


population_size = 4
num_generations = 10

# Instantiate the initializers from json file
toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list, toolbox.individual_guess, JSON_PATH+"vgg16.json")

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', train_evaluate)

population = toolbox.population_guess()
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, 
ngen = num_generations, verbose = False)


<class 'deap.creator.Individual'>
['l', 'a', 'y', 'e', 'r', 's']
Validation Fitness Function:  1 

<class 'deap.creator.Individual'>
['n', 'a', 'm', 'e']
Validation Fitness Function:  2 



TypeError: list indices must be integers or slices, not str

In [ ]:
# Print top N solutions
best_individuals = tools.selBest(population,k = 2)
print('\nMost fit are: ')
for bi in best_individuals:
    print('\n me')

In [ ]:
# Train the model using best configuration on complete training set 
#and make predictions on the test set
X_train, y_train, X_test, y_test = load_cifar10_data(test_data,best_window_size)

inputs = Input(shape=(best_window_size,1))
x = LSTM(best_num_units, input_shape=(best_window_size,1))(inputs)
predictions = Dense(1, activation='linear')(x)
model = Model(inputs = inputs, outputs = predictions)
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X_train, y_train, epochs=5, batch_size=10,shuffle=True)
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test RMSE: ', rmse)